In [42]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk import ngrams
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
from transformers import pipeline

## Feladatok

### 1. Adatok betöltése és előfeldolgozás

- Töltsd be a Twitter vélemények adathalmazt Pandas segítségével.
- Ellenőrizd, hogy milyen oszlopok vannak benne és hány rekordot tartalmaz.
- Nézd meg az első 5 sort.

link: https://raw.githubusercontent.com/sharmaroshan/Twitter-Sentiment-Analysis/refs/heads/master/train_tweet.csv

In [43]:
url = "https://raw.githubusercontent.com/sharmaroshan/Twitter-Sentiment-Analysis/refs/heads/master/train_tweet.csv"

df = pd.read_csv(url, sep=",", header=0, encoding='latin1', on_bad_lines='skip')

print(df.info())

display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB
None


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


### 2. Szövegtisztítás és tokenizálás
- Tisztítsd meg a Twitter szövegeket!
- Távolítsd el a speciális karaktereket, URL-eket és számokat.
- Tokenizáld a mondatokat szavakra.

In [44]:
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download('punkt_tab')
import re

text = df.tweet[0]

stop_words = set(stopwords.words("english"))

clean_text = re.sub( r'@|#', '', text)

words = word_tokenize(clean_text)

words_eredeti = word_tokenize(text)

filtered_words = [word for word in words if word.lower() not in stop_words]

print("\nEredeti szólista:", words_eredeti[:20])
print("\nTisztított:", filtered_words[:20])


Eredeti szólista: ['@', 'user', 'when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#']

Tisztított: ['user', 'father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', '.', 'run']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 3. Szövegek hossza, histogram
- Határozd meg hogy melyik szöveg milyen hosszú.
- Számold ki, hogy mennyi az átlagos hossza a pozitív és a negatív üzeneteknek.
- Készíts egy-egy histogramot a szövegek hosszának gyakoriságáról a pozitív és a negatív tweetek esetén.

In [45]:
import seaborn as sns


df['clean_text'] = clean_text

df['text_length'] = df['clean_text'].apply(lambda x: len(x.split()))



nltk.download("vader_lexicon")


sia = SentimentIntensityAnalyzer()

df["sentiment_score"] = df["clean_text"].apply(lambda x: sia.polarity_scores(x)["compound"])

df["sentiment"] = df["sentiment_score"].apply(lambda x: "positive" if x > 0.05 else ("negative" if x < -0.05 else "neutral"))


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### 4. Szavak vizualizálása (WordCloud)
- Készíts egy WordCloud-ot a leggyakoribb pozitív és negatív szavakról.

In [46]:
plt.figure(figsize=(12, 6))

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

### 5. BERT használata véleménydetekcióra
- Használj Hugging Face BERT modellt a Twitter vélemények osztályozására.
- Próbálj ki egy pozitív és egy negatív szöveget

In [47]:
sentiment_model = pipeline("sentiment-analysis")

sample_text = df["clean_text"].iloc[0]
result = sentiment_model(sample_text)

print(f"\nSMS: {sample_text}")
print(f"Predikció: {result}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



SMS:  user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   run
Predikció: [{'label': 'NEGATIVE', 'score': 0.9994018077850342}]
